# Gummy Bears
I picked up a package of gummy bears which was called [_12 Flavor Gummi Bears_](https://www.albanesecandy.com/12-flavor-gummi-bears-r/).

![12 Flavor Gummi Bears](https://cdn11.bigcommerce.com/s-riqk6cih6h/images/stencil/640w/products/534/1294/12-flavor-gummi-bears_1oz__59308.1626975777.jpg?c=1)

On the back of the package, there was a statement: 

> DUE TO SIZE OF BAG,
>
> MAY NOT CONTAIN
>
> ALL 12 FLAVORS

That got me thinking: What was the probability that a given bag _did_ contain all 12 flavors?

## Problem statement
Let's first be clear what we're asking. We'll define the **_Gummy Bear Problem_** as follows:

> A package has $n$ gummy bears. There are $m$ distinct flavors (or types) of gummy bears ($n \geq m$). What is the probability that a package has _at least_ one of each flavor of gummy bear, i.e. that a package contains all possible flavors? 
> 
> Assume that when gummy bears are put into the package, each flavor of gummy bear has the same probability of being chosen, and that the selection of one gummy bear has no impact on the selection of other gummy bears.

# Approach to the problem
There are at least two ways to approach this problem:

1. **Numerical approximation**: Run a simulation many times, and use the average results to come up with an approximation or _estimate_ of the solution.
2. **Analytical solution**: Use some theory to come up with an expression that gives us the _exact_ answer.

Let's look at each.

# Numerical Approximation through simulation: Monte Carlo methods

An approximate numerical solution can be easier to derive because it requires less understanding of probability theory. Furthermore, not all problems even have an analytical solution, but for those that do, the analytical solution is often more difficult to reason about.

One numerical approach is using a simulation to estimate the solution. Let's code up a simulation where we will:

1. Generate a package of $n$ gummy bears using $m$ equally-probable flavors/types.
2. Determine if the package has all $m$ flavors.
3. Do steps (1) and (2) many times, and count the number of times the generated package had all flavors.
4. The fraction of times the package had all flavors will be our estimate of the probability.

The main thing we need here is a source of randomness, which will be used in Step 1 above to generate a gummy bear package in the following way:

1. In Python, we can use `random.randrange(m)` to generate a number between $[0, m - 1]$ which represents a gummy bear flavor. Each of the $m$ values thus has an equal chance of being generated each time we call this method.
2. We then do this $n$ times and store each value in a list, which then represents a single gummy bear package or _sample_. We can do this because we assumed that gummy bears were selected independently, i.e. the previous flavor selections do not affect the subsequent selections.

This can be done many times, after which we count the fraction of packages that had all flavors. The advantage of a numerical approximation like this is that it's relatively simple to understand, and doesn't require any special understanding of probability theory other than the assumption we've made that all flavors are equally probable.

In [1]:
# Imports
import random
import statistics
import time

import matplotlib.pyplot as plt

In [6]:
# Helper functions
# NOTE: This code is not necessarily optimized for speed, but instead for readability.

# A gummy bear package is represented as a list of integers.
# Each value will be in the range [0, num_types - 1]
# This method returns a randomly-generated package of gummy bears assuming each flavor/type is equally-probably.
def generate_gummy_bear_package_sample(package_size, num_types):
  # Use the integers [0, num_types - 1] to represent a gummy bear type/flavor.
  # random.randrange(num_types) will return a integer values in the range with equal probability.
  return [random.randrange(num_types) for i in range(package_size)]

# Returns the number of unique integer values in a list of integers.
# This represents the number of unique flavors in the package.
def get_num_unique_gummy_bear_types(gummy_bear_package):
  # Dump the list into a set and get its size.
  return len(set(gummy_bear_package))

# We define one sample as one gummy bear package.
# This generates the specified number of samples by generating a gummy bear package for each sample and
# getting the number of unique flavors in that package.
def generate_samples(num_samples, package_size, num_types):
  # Generate a package of gummy bears for each sample, and return the number of unique types in each package.
  # Each entry in the returned list is the number of unique types in the package generated for that samples.
  return [get_num_unique_gummy_bear_types(generate_gummy_bear_package_sample(package_size, num_types)) for i in range(num_samples)]

# We define a single trial (experiment) as:
# 1. Generating a certain number of gummy bear packages (num_samples)
# 2. Determining the fraction of those gummy bear packages that contained at least one of each flavor.
# The fraction forms an estimate of the true probability.
def run_single_trial(num_samples, package_size, num_types, print_results=False):
  # A single trial will generate the specified number of gummy bear packages (sample size) and calculate the estimated probability from the results.
  samples = generate_samples(num_samples, package_size, num_types)
  num_packages_with_all_types = len([s for s in samples if s == num_types])
  estimated_probability = num_packages_with_all_types/num_samples
  if print_results: # For verbose output.
    print(f'Out of {num_samples} samples (generating a package of gummy bears), {num_packages_with_all_types} packages contained all flavors for a fraction of {estimated_probability:.8f}')
  return estimated_probability

# Convenience method for running multiple trials.
def run_trials(num_trials, num_samples, package_size, num_types):
  start = time.time()
  # Run a number of trials and return the estimated probability from each.
  estimates = []
  for i in range(num_trials):
    estimates.append(run_single_trial(num_samples, package_size, num_types))
  print_estimates(estimates, package_size, num_types)
  print(f'Time taken: {(time.time() - start):.2f} seconds')
  return estimates

# Convenience method for printing out the results.
def print_estimates(estimates, n, m):
  print(f'Results for n={n}, m={m}')
  print([f'{e:.8f}' for e in estimates])
  print(f'mean: {statistics.mean(estimates):.8f}')

Let's run a simulation of a few trials for $n = 12$ (number of gummy bears in a package) and $m = 12$. (number of gummy bear flavors)

In this case, we're calculating an _estimate_ of the probability of a package of 12 gummy bears containing exactly all 12 flavors.

In [7]:
num_samples = 1000000
num_trials = 10

In [8]:
# For n = 12, m = 12
estimates = run_trials(num_trials, num_samples, package_size=12, num_types=12)

Results for n=12, m=12
['0.00004900', '0.00005600', '0.00005000', '0.00005000', '0.00005300', '0.00006900', '0.00004700', '0.00005400', '0.00005300', '0.00005600']
mean: 0.00005370
Time taken: 124.11 seconds


The estimated probability here is very low, as a probability of 0.00005 is about a 1 in 20,000 chance.

Let's also run simulations for $n = \{13, 14, 15, 25\}, m = 12$:

In [9]:
# For n = 13, m = 12
estimates = run_trials(num_trials, num_samples, package_size=13, num_types=12)

Results for n=13, m=12
['0.00036500', '0.00031700', '0.00037500', '0.00033600', '0.00037600', '0.00036400', '0.00034400', '0.00036200', '0.00036000', '0.00035500']
mean: 0.00035540
Time taken: 137.21 seconds


For a package size of 13 (one more than the number of flavors), the estimated probability is improved quite a bit, but is still very low. A probability of 0.00035 is about a 1 in 2900 chance.

In [10]:
# For n = 14, m = 12
estimates = run_trials(num_trials, num_samples, package_size=14, num_types=12)

Results for n=14, m=12
['0.00128300', '0.00129100', '0.00124800', '0.00123100', '0.00121400', '0.00120100', '0.00122900', '0.00124400', '0.00121600', '0.00125200']
mean: 0.00124090
Time taken: 144.67 seconds


For a package size of 14, the chances are about 1 in 800.


In [11]:
# For n = 15, m = 12
estimates = run_trials(num_trials, num_samples, package_size=15, num_types=12)

Results for n=15, m=12
['0.00326500', '0.00329900', '0.00332500', '0.00334700', '0.00333300', '0.00330500', '0.00330500', '0.00337100', '0.00331100', '0.00331900']
mean: 0.00331800
Time taken: 153.03 seconds


With a package size of 15, the chances are further improved to about 1 in 300.

In [12]:
# For n = 25, m = 12
estimates = run_trials(num_trials, num_samples, package_size=25, num_types=12)

Results for n=25, m=12
['0.18147000', '0.18225200', '0.18248600', '0.18188100', '0.18221900', '0.18172500', '0.18191700', '0.18214600', '0.18178800', '0.18222400']
mean: 0.18201080
Time taken: 246.21 seconds


With a package size of 25, the estimated probability is something that's reasonably frequent: About 18% or a little better than a 1 in 6 chance of getting all 12 flavors.

# Analytical Solution
A numerical approximation (here based on a simulation, though not all numerical methods are based on simulation) is fairly easy to code, but determining an analytical (closed-form) solution will give us the exact answer, and generally "feels" nicer. However, this requires a bit more effort.

The first thing to realize is that our gummy bear package represents a sample from the [_multinomial distribution_](https://en.wikipedia.org/wiki/Multinomial_distribution). Such a distribution represents the potential outcomes from conducting $n$ trials, each of which can produce one of $m$ different results or categories. The outcome is then the counts for each of the $m$ categories after $n$ trials.

I've [written about the multinomial distribution](https://peterchng.com/blog/2020/10/23/building-binomial-and-multinomial-samplers-in-java/) before, but basically with a package size of $n$ gummy bears and $m$ unique flavors, each of the gummy bears represents a trial where one of $m$ flavors is selected.

Thus, we want to calculate the probability that a sample from this multinomial distribution contains at least one of each gummy bear flavor.

## Definitions
Let's clarify the variables we'll use in the following sections:

- $n$: The number of gummy bears per package (number of trials in a multinomial distribution)
- $m$: The number of different flavors of gummy bears (number of categories in a multinomial distribution)
- $X_i$: The random variable representing the count of the number of gummy bears of flavor $i$ and $i \in [1, m]$
- $x_i$: The actual count of gummy bears for flavor $i$
- $p_i$: The probability of flavor $i$. Since we assumed all flavors are equally probable, all of these are equal, i.e. $p_i = p = 1/m$


## Solving for the special case of $n = m$
Before coming up with a general approach, let's look at the specific case where the package size of gummy bears is exactly equal to the number of flavors, i.e. $n = m$.

In this case, in order to get all flavors in the package, you have to get one (and exactly one) of each type of gummy bear in the package. We can use the probability mass function (PMF) of the multinomial distribution to compute the exact probability of this.

The PMF of the multinomial distribution is defined as:

$$
f(x_1, ... x_m; n, p_1, ... p_m) = P(X_1 = x_1, ..., X_m = x_m)
$$

$$
= {n!\over{x_1! \cdots x_m!}}p_1^{x_1}...p_m^{x_m}
$$

It is written in terms of the number of each flavor of gummy bear of we expect ($x_i$) and the probabilities associated with each ($p_i$), (i.e. gummy bear flavor $x_i$ has probability $p_i$ of being chosen), and the number of gummy bears in each package, $n$.

Plugging in values for $n = m = 12$ and our assumption that all flavors are equally probable, i.e. $p_i = p = 1/12$ yields the expression:

$$
{12!\over{1! \cdots 1!}}\left(1\over{12}\right)^{12}
$$

This reduces to:

$$
{12!\over{12^{12}}} \approx 0.0000537
$$

This value closely aligns with the estimation we previously obtained through our numerical or simulation approach.

To get an intuitive understanding why this expression is correct, we can rewrite the above expression as:

$$
{12\over{12}}\times{11\over{12}}\times \cdots \times {1\over{12}}
$$

This can be explained as:
1. Imagine assembling the package of 12 gummy bears one bear at a time.
2. For the first gummy bear, you can pick any flavor, which represents a probability of 100% or $12/12$.
3. For the next gummy bear, since we can only have one of each type, we only have 11 choices out of 12, so the probability is reduced to $11/12$.
4. The probability for each subsequent choice is reduced by the same fraction, until the last gummy bear, where you must pick the last flavor that hasn't yet been picked, yielding a probability of $1/12$.


## Solving for specific cases where $n \gt m$

When the $n \gt m$, i.e. the package size is larger than the number of unique flavors, determining an analytical solution to the problem is trickier because there is more than one outcome that satisfies our requirement of "at least one gummy bear of each flavor".

It's obvious that as we increase the package size, the probability of getting all flavors will increase, since we have more chances to get at least one of each flavor. That is, we should expect $P(\text{get all flavors} \: | \: n > m) \gt P(\text{get all flavors} \: | \: n = m)$. However, exactly _how_ the probability increases isn't obvious.

Let's look at a few specific cases of $n > m$ to try and figure out the pattern for a general approach.

### Solution for $n = 13, m = 12$

As a first attempt to tackle this, let's increase $n$ by one to $n = 13$ and try to solve for this specific case. Because there are 13 gummy bears per package, you can have _at most_ 2 of the same flavor and still have all 12 flavors. That is, all the valid configurations consist of two gummy bears of one flavor, and exactly one gummy bear for the remaining 11.

Because there are 12 gummy bear flavors, there are 12 such configurations, and each has the same probability because we previously assumed all gummy bears were equally probable. We can then calculate the solution as:

$$
12 \times P(\text{get 2 of one flavor and 1 of each remaining flavor})
$$

The probability term can again be calculated using the multinomial PMF. Plugging in values, we get this solution:

$$
= 12 \times {13!\over{2!}}\left(1\over{12}\right)^{13} \approx 0.00034920091
$$

This value again aligns closely with the estimates we obtained above using a numerical approximation.


### Solution for $n = 14, m = 12$

In this case, after ensuring each flavor gets one gummy bear assigned to it, there are two (2) extra gummy bears to distribute. Any way that these 2 extra gummy bears are distributed across the 12 flavors is a valid package configuration. Thus, computing the probability is reduced to the following steps:

1. How many ways (or _configurations_) are there to distribute the extra gummy bears across the flavors?
2. What is the probability of each of those configurations?
3. The solution is then the sum of these probabilities.

Step (1) is a _combinatorics_ problem, and step (2) can be found using the multinomial PMF as before.

If you're familiar with combinatorics (I am not), there is an approach called [_Stars and Bars_](https://en.wikipedia.org/wiki/Stars_and_bars_%28combinatorics%29) that can be used to solve problems like "_how many ways are there to distribute $n$ balls across $k$ bins_?"

At first I thought this approach could be used to solve the problem, unfortunately by itself, it cannot. While this method will tell us how many ways we can distribute the 2 remaining gummy bears across the 12 flavors, not all of these configurations have the _same_ probability. So, we have to look at each configuration individually.

For 2 extra gummy bears, there are **two configurations** of how these could be distributed across the 12 flavors:

1. The 2 extra gummy bears go to a _single_ flavor
2. One extra gummy bear goes to one flavor, and one goes to another _different_ flavor.

Because the counts for each flavor differ here, the probability for each of these configurations differ due to the factorial term in the multinomial PMF. Recall its form:

$$
{n!\over{x_1! \cdots x_m!}}p_1^{x_1}...p_m^{x_m}
$$

Now, we have to figure out how many ways we can obtain each of the two configurations:

- For (1), how many ways are there to have the extra 2 gummy bears going to a single flavor?
Because there are 12 flavors, this is simply 12. (You can also view it as ${12}\choose{1}$ because we are picking 1 flavor of the 12)
- For (2), how many ways are there to have the 2 extra gummy bears go to two different flavors?
Following our previous reasoning, because there are 12 flavors, this is ${12}\choose{2}$.

We can add up these two numbers and check the total number of combinations against the [stars and bars](https://en.wikipedia.org/wiki/Stars_and_bars_%28combinatorics%29) method. 

- Adding up the two values yields: ${{12}\choose{1}} + {{12}\choose{2}} = 78$. 
- Using stars and bars to find out how many ways to distribute 2 gummy bears over 12 flavors yields the expression ${{2 + 12 - 1}\choose{12 - 1}} = {{13}\choose{11}} = 78$.

So, our reasoning seems correct.

We can now write the solution as:

$$
{{12}\choose{1}} \times P(\text{A}) + {{12}\choose{2}} \times P(\text{B})
$$

where

- $A$: The two extra gummy bears go to a single flavor.
- $B$: The two extra gummy bears go to different flavors.

This evaluates to:

$$
= {{12}\choose{1}} \times {14!\over{3!}}\left(1\over{12}\right)^{14} + {{12}\choose{2}} \times {14!\over{2!2!}}\left(1\over{12}\right)^{14}
$$

$$
\approx 0.00125615327
$$

This value is again close to our estimate from the numerical approach.

### Solution for $n = 15, m = 12$

In this case, after ensuring each flavor gets one gummy bear, there are 3 remaining gummy bears to distribute. Let's use our prior approach to solve this one.

How many configurations are there? We can break down the number 3 in the following manner:

1. The $(3)$ configuration: All 3 go to one flavor.
2. The $(2, 1)$ configuration: 2 goes to one flavor, 1 goes to another.
3. The $(1, 1, 1)$ configuration: 1 goes to one flavor, 1 goes to another, and 1 goes to yet another.

For (1): How many ways are there such that all 3 go to one flavor? This is again ${{12}\choose{1}} = 12$.

For (2): This is a bit tricky, but let's break it down this way:
- The first two have to go to different flavors, which we'll call $a$ and $b$. From before, we know this is ${{12}\choose{2}} = 66$
- The third one can go to flavor $a$ or $b$. Because there are two flavors here, this is equivalent to ${{2}\choose{1}} = 2$
- Thus there are ${{12}\choose{2}}{{2}\choose{1}} = 132$ different ways for the $(2, 1)$ configuration.

For (3): This is equal to ${{12}\choose{3}} = 220$ different ways for the $(1, 1, 1)$ configuration.

The total number of ways is thus $12 + 132 + 220 = 364$. Using the stars and bars method to check, we find that there are also ${{3 + 12 - 1}\choose{12 - 1}} = {{14}\choose{11}} = 364$ ways to distribute 3 flavors across 12 gummy bears, so our approach seems correct.

Calculating the sum of the probabilities of these configurations yields the expression:

$$
= 12 \times {15!\over{4!}}\left(1\over{12}\right)^{15} + 132 \times {15!\over{3!2!}}\left(1\over{12}\right)^{15} + 220 \times {15!\over{2!2!2!}}\left(1\over{12}\right)^{15}
$$

$$
\approx 0.00331013362
$$

## A general approach for $n > m$
After solving for a few specific cases, each with increasing complexity as the value of $(n - m)$ increases, we can begin to draft a general approach that uses both combinatorics and probability:

1. The number $(n - m)$ represents the remaining (or extra) gummy bears after each flavor has gotten one gummy bear assigned. These can be distributed in any way across the $m$ flavors.
2. Determine the **valid configurations** by doing an [integer partition](https://en.wikipedia.org/wiki/Partition_(number_theory)) of the value $(n - m)$.
  - a. For example the number $3$ can be written as a sum of positive integers in the following way: $(3)$, $(2 + 1)$, and $(1 + 1 + 1)$.
  - b. These are the configurations of how the extra gummy bears can be distributed across the flavors.
  - c. Each of these configurations will have a separate probability that can be calculated from the multinomial PMF.
3. For each configuration, determine the **number of ways that configuration can be achieved**. This is the trickiest part!
4. Multiply the number of ways for each configuration (3) by the probability for each configuration (2c) and sum the results - this will be the exact solution.

This approach gets tedious as the number of extra gummy bears increases, which is why I did not attempt to solve this analytically for $n = 25$. Unfortunately, I could not find a better way for a general approach.


## An alternative approache using the multinomial CDF?

The probability of getting a package of size $n$ with all $m$ gummy bear flavors can be expressed this way:

$$
P(X_1 \geq 1, X_2 \geq 1, ...,  X_{m} \geq 1) \hspace{5mm} \text{when} \hspace{5mm} \sum_{i = 0}^{m} x_i = n
$$

That is, we're looking for the probability that we get at least one of each flavor in a package of $n$ gummy bears.

This is _almost_ the complement of the cumulative distribution function (CDF). The multinomial distribution is a multivariate distribution (the samples are vectors rather than scalars) and a multivariate CDF has the form:

$$
F(x_1, x_2, ..., x_m) = P(X_1 \leq x_1, X_2 \leq x_2, ...,  X_m \leq x_m)
$$

So, if we could determine the CDF for the multinomial distribution, we could probably work out an expression for the solution to our problem. However, CDFs for multivariate distributions tend to be complicated, and it wasn't obvious to me how to compute the CDF other than through a naive brute-force approach of summing up all the appropriate PMF values.

After doing a bit of searching, I found [this 1981 paper](https://projecteuclid.org/journals/annals-of-statistics/volume-9/issue-5/A-Representation-for-Multinomial-Cumulative-Distribution-Functions/10.1214/aos/1176345593.full) from _The Annals of Statistics_ by [Bruce Levin](https://www.publichealth.columbia.edu/people/our-faculty/bl6) which offers a closed-form representation of the multinomial CDF. It expresses the multinomial distribution as a conditional distribution of independent Poisson random variables subject to a fixed sum, and uses that to derive an expression for the multinomial CDF.

This could be used to solve the Gummy Bear Problem here, though I have not done so, since the CDF also involved calculating the probability for a sum of independent right-truncated Poisson RVs, which I haven't figured out how to do yet. (See this [Ph.D disseration from 1968](https://dr.lib.iastate.edu/entities/publication/ec3f45af-ff35-40a5-82fe-3ecbf2d803a2) for more on truncated Poisson distributions)

# Conclusion

Determining a solution to this Gummy Bear Problem can get complicated depending on the approach, despite the problem being simple to state.

For a numerical approximation, the solution is fairly straightforward: Write some code using a source of randomness, then run a bunch of simulations and collect the results. We have to verify the code for correctness, but we don't really have to understand much probability theory or combinatorics. However, this solution only gives us an estimate. We can improve the accuracy of the estimation by running more simulations and taking more CPU time.

For the analytical solution, we have to apply both combinatorics to figure out the number of valid package configurations, and also probability theory in the form of the multinomial distribution to calculate the probability of each of those valid configurations. We can then come up with an expression for the solution, which may still be quite involved! However, the overall calculation isn't as computationally complex as the simulation done for the numerical approximation, and will give us the _exact_ answer rather than an approximation. Furthermore, understanding the analytical solution may give you a deeper appreciation for the problem.

We also simplified this problem by making certain assumptions: Equal probability across all flavors and independence between gummy bear selection. Removing these assumptions would make the numerical solution slightly more complex to code, but also make the analytical solution much more tougher to reason about.

Most real-world problems are much more complex than the Gummy Bear Problem presented here, and may not even have analytical (closed-form) solutions. This is why numerical analysis methods are so commonplace!

**And for the record**: The package of gummy bears I got contained 13 gummy bears, and I _did not_ get all 12 flavors.